In [39]:
# Submission - 1
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import glob
import statistics
import os

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Specify the file path
directory_path = 'C:\\Abhishek\\IIITD\\Academics\\CSE\\CSE558_DSc\\Project\\PRSA_Data_20130301-20170228'
directory_path2 = 'C:\\Abhishek\\IIITD\\Academics\\CSE\\CSE558_DSc\\Project'


First we combine all the csvs into one, keeping a column for the weather station name separately

In [40]:
# Get a list of all files in the directory
file_list = os.listdir(directory_path)

# Combined dataframe
com_df = []

# Iterate through the files in the directory
for filename in file_list:
    # Construct the full file path
    file_path = os.path.join(directory_path, filename)
    
    # Check if the current item is a file
    if os.path.isfile(file_path):
        "Reading", filename
        df = pd.read_csv(file_path, index_col=None, header=0) # Converting the file to a dataframe
        com_df.append(df)
data = pd.concat(com_df)
data.to_csv('output.csv', index=False)

Then for the sake of uniformity we rename the columns so it remains the same for everyone

In [41]:
data.drop(["No"], axis=1, inplace=True)
data.rename(columns = {'year': 'Year', 'month': 'Month', 'day': "Day", 'hour': 'Hour', 'pm2.5': 'PM2.5', 'DEWP': 'DewP', 'TEMP': 'Temp', 'PRES': 'Press', 'RAIN': 'Rain','wd': 'WinDir', 'WSPM': 'WinSpeed', 'station': 'Station'}, inplace = True)
data.to_csv('dataset.csv', index=False)

We also need to deal with the null and missing values present in this dataset

In [42]:
data.isnull().sum()

Year            0
Month           0
Day             0
Hour            0
PM2.5        8739
PM10         6449
SO2          9020
NO2         12116
CO          20701
O3          13277
Temp          398
Press         393
DewP          403
Rain          390
WinDir       1822
WinSpeed      318
Station         0
dtype: int64

This will be done as follows :
1. For continuous attributes like CO levels etc., collate data for each month and simply replace NA with the average value.
2. For categorical attributes like wind direction, collate the data from the entire dataset and replace with the mode value.

In [43]:
directory_path2 = 'C:\\Abhishek\\IIITD\\Academics\\CSE\\CSE558_DSc\\Project'
df = pd.read_csv(os.path.join(directory_path2, 'dataset.csv'), index_col=None)

# Define the columns for which you want to calculate averages
columns_to_average = ['PM2.5','PM10','SO2','NO2','CO','O3','Temp','Press','DewP','Rain','WinSpeed'] # Removing Wind direction since we want to use mode for that

# Define the columns with missing values
columns_with_missing = ['WinSpeed']

# Group the DataFrame by 'month' and calculate the averages for the specified columns
averages_by_month = df.groupby('Month')[columns_to_average].mean()

# Group the DataFrame by 'month' and calculate the mode for the categorical column
modes_by_month = df.groupby('Month')[['WinDir']].agg(lambda x: x.mode()[0] if not x.mode().empty else None)

# Print the resulting DataFrame
print(averages_by_month)
print(modes_by_month)

            PM2.5        PM10        NO2           CO         O3       Temp  \
Month                                                                         
1       93.667037  113.227543  61.763499  1852.933132  27.633733  -1.797300   
2       87.572234  102.682735  50.213573  1384.996511  40.734810   1.123071   
3       94.660678  136.040075  60.124179  1310.230474  51.041261   8.348409   
4       72.734890  116.962595  47.280261   840.493265  70.185707  15.328909   
5       63.105338  107.038514  42.534000   795.234500  92.028567  21.482056   
6       69.091971   86.730518  39.319569   987.904479  94.692310  24.554861   
7       71.744853   84.631258  35.822804   891.855104  95.086942  27.194170   
8       53.473014   71.511467  36.103815   825.603120  87.871592  26.642028   
9       61.478105   79.673490  45.616774   925.164610  55.094877  20.765633   
10      91.726800  112.550553  59.738056  1150.505966  29.927943  13.450143   
11      93.331519  118.113954  61.895454  1660.20485

Now we actually replace the values based on the averages calculated

In [49]:
# Iterate through the DataFrame and replace null or missing values with the corresponding averages

for column in df.columns:
    print(column)
    
i = 0
for index, row in df.iterrows():
    if i > 5:
        break
    print(df.at[index,'SO2'])
    i += 1

# for index, row in df.iterrows():
#     for column in df.columns:
#         if pd.isnull(df.at[index, column]):
#             if column == 'WinDir':
#                 df.at[index, column] = modes_by_month.loc[df.at[index, 'Month'], 'WinDir']
#             else:
#                 df.at[index, column] = averages_by_month.loc[df.at[index, 'Month'], column]

# print(df[:10])

Year
Month
Day
Hour
PM2.5
PM10
SO2
NO2
CO
O3
Temp
Press
DewP
Rain
WinDir
WinSpeed
Station
4
4
5
11
12
18
